In [1]:
# Imports
import gymnasium as gym
from typing import Callable

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecMonitor
import torch as th

import pickle

# Import Our environment
from dev_env import tradingEng


c:\Users\osc16\miniconda3\envs\d2d\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Load Paths
with open("1.6kRunDemo.pkl","rb") as fp:
    paths = pickle.load(fp)

paths1 = paths[0:300]
paths2 = paths[300:600]
paths3 = paths[600:900]
paths4 = paths[900:1200]
del(paths)

In [3]:
## LR schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

import numpy as np
  ## LR schedule
def exp_schedule(initial_value: float, decayRate = 5) -> Callable[[float], float]:
    """
    Exp learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        factor = 1 - progress_remaining
        return (np.exp((1-factor)*decayRate) - 1) * initial_value / (np.exp(decayRate) - 1)

    return func

In [4]:
# Def Env
def start_and_release(set, action = 'small-More-Trust', obs = 'big'):
    ret = tradingEng(set, action = action, obs = obs, reward='L2')
    #del(set)
    return ret

#t = start_and_release(paths1,action='small-More-Trust', obs = 'auto')
envs = VecMonitor(DummyVecEnv([
    lambda: start_and_release(paths1,action = 'small-More-Trust', obs = 'xs',),
    lambda: start_and_release(paths2,action = 'small-More-Trust', obs = 'xs',),
    lambda: start_and_release(paths3,action = 'small-More-Trust', obs = 'xs',),
    lambda: start_and_release(paths4,action = 'small-More-Trust', obs = 'xs'), #4
]))

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam))
model = PPO("MlpPolicy", envs, batch_size = 652*2, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=652*4, normalize_advantage=True, gamma = 0.9, verbose = 1)

model.learn(total_timesteps=int(3e6), log_interval=1)
# Save the agent
model.save("PPOs")

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 653      |
|    ep_rew_mean     | -0.0302  |
| time/              |          |
|    fps             | 1884     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 10432    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 652          |
|    ep_rew_mean          | -0.032       |
| time/                   |              |
|    fps                  | 1440         |
|    iterations           | 2            |
|    time_elapsed         | 14           |
|    total_timesteps      | 20864        |
| train/                  |              |
|    approx_kl            | 0.0026332946 |
|    clip_fraction        | 0.00347      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.84        |
|    explained_variance   | -3.9         

In [14]:
import numpy as np
print(env._get_obs())
path = paths[800]
lambdax = path.lambdas[25]
r = path.r[25]
t = path.t_s[25]
print(lambdax)
print(r)

#model.predict(observation=np.asmatrix([lambdax,r,t]),state=None, deterministic=True)

[[-0.99977158  0.05536838 -0.56841282]]
0.005196713067477058
0.030745992173113156
